<a href="https://colab.research.google.com/github/sushil79g/nepali_caption/blob/master/notebooks/nepali_caption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
base_dir = 'drive/My Drive/Image_Captioning/'

In [0]:
import pandas as pd
train_nepali = pd.read_csv(base_dir + 'annotations/caption_train_withNepali.csv')
validation_nepali = pd.read_csv(base_dir + 'annotations/caption_validation_withNepali.csv',names=['image_id','id','en_caption','ne_caption'])
train_nepali.drop('Unnamed: 0',inplace=True,axis=1)
train_nepali.columns = ['image_id','id','en_caption','ne_caption']

In [0]:
validation_nepali.head()

,image_id,id,en_caption,ne_caption
0,203564,37,A bicycle replica with a clock as the front wh...,अगाडि पाङ्ग्रा रूपमा घडी संग एक साइकल नक्कली।
1,179765,38,A black Honda motorcycle parked in front of a ...,कालो होन्डा मोटरसाइकल एक ग्यारेज अगाडि पार्क।
2,322141,49,A room with blue walls and a white sink and door.,नीलो पर्खाल संग एक कोठा र एउटा सेतो डुब्नु र ढ...
3,16977,89,A car that seems to be parked illegally behind...,एक कानुनी रूपमा पार्क कार पछि अवैध पार्क देखिन...
4,106140,98,A large passenger airplane flying through the ...,एक ठूलो यात्री हवाइजहाज हावा मार्फत उडान।


In [0]:
import json
with open(base_dir + 'annotations/captions_train2014.json') as outfile:  
    train_json = json.load(outfile)

In [0]:
for i in range(5):
    print(train_json['annotations'][i])

{'image_id': 318556, 'id': 48, 'caption': 'A very clean and well decorated empty bathroom'}
{'image_id': 116100, 'id': 67, 'caption': 'A panoramic view of a kitchen and all of its appliances.'}
{'image_id': 318556, 'id': 126, 'caption': 'A blue and white bathroom with butterfly themed wall tiles.'}
{'image_id': 116100, 'id': 148, 'caption': 'A panoramic photo of a kitchen and dining room'}
{'image_id': 379340, 'id': 173, 'caption': 'A graffiti-ed stop sign across the street from a red car '}


In [0]:
# for item in train_json['annotations']:
#     image_id_train = item['image_id']
#     id_train = item['id']
#     caption_en = item['caption']
#     print(caption_en)
#     break

एक धेरै सफा र राम्रो सजाया खाली बाथरूम


In [0]:
# temp_ = train_nepali[train_nepali.image_id == 116100]
# caption = list(temp_[temp_.id==148]['ne_caption'])[0]
# caption

'एक भान्सा र भोजन कक्ष को एक मनोरम फोटो'

In [0]:
for index,item in enumerate(train_json['annotations']):
    image_id_train = item['image_id']
    id_train = item['id']
    temp_ = train_nepali[train_nepali.image_id == image_id_train]
    caption = list(temp_[temp_.id==id_train]['ne_caption'])[0]
    item['caption'] = caption
    

In [0]:
import json
with open(base_dir + 'annotations/captions_val2014.json') as outfile:  
    val_json = json.load(outfile)

In [0]:
for i in range(5):
    print(val_json['annotations'][i])

{'image_id': 203564, 'id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.'}
{'image_id': 179765, 'id': 38, 'caption': 'A black Honda motorcycle parked in front of a garage.'}
{'image_id': 322141, 'id': 49, 'caption': 'A room with blue walls and a white sink and door.'}
{'image_id': 16977, 'id': 89, 'caption': 'A car that seems to be parked illegally behind a legally parked car'}
{'image_id': 106140, 'id': 98, 'caption': 'A large passenger airplane flying through the air.'}


In [0]:
from tqdm import tqdm
for index,item in tqdm(enumerate(val_json['annotations'])):
    image_id_val = item['image_id']
    id_val = item['id']
    temp_ = validation_nepali[validation_nepali.image_id == image_id_val]
    caption = list(temp_[temp_.id==id_val]['ne_caption'])[0]
    item['caption'] = caption

202654it [05:39, 596.90it/s]


In [0]:
with open('train_nepali.json', 'w', encoding='utf-8') as f:
    json.dump(train_json, f)

In [0]:
with open('validation_nepali.json', 'w', encoding='utf-8') as f:
    json.dump(val_json, f)

In [23]:
len(train_nepali.image_id.unique())

82783

In [24]:
len(validation_nepali.image_id.unique())

40504

In [0]:
##lets start modelling

In [0]:
import os
import sys
sys.path.append('/opt/cocoapi/PythonAPI/')
from pycocotools.coco import COCO

In [0]:
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
##get the images for training 
!wget http://images.cocodataset.org/zips/train2014.zip

--2019-07-18 07:58:24--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.145.3
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.145.3|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  92.2MB/s    in 3m 22s  

2019-07-18 08:01:46 (63.9 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]



In [25]:
#get the image for validation
!wget http://images.cocodataset.org/zips/val2014.zip

--2019-07-18 08:03:15--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.37.100
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.37.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[===================>]   6.19G  65.4MB/s    in 2m 24s  

2019-07-18 08:05:39 (44.0 MB/s) - ‘val2014.zip’ saved [6645013297/6645013297]



In [0]:
!unzip -q train2014.zip

In [0]:
!unzip -q val2014.zip